### ספריות


In [1]:
import os
import sys
import pandas as pd
from shutil import copy
from openpyxl import load_workbook
from functions import change_Muni_Heb_to_Muni_Eng

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

### העלת משתנים להרצת הקוד


In [4]:
cwd = os.getcwd()
software_data_folder_location = os.path.dirname(cwd)
df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_data_folder_location))

In [5]:
create_forecast_basic_location=df_inputs_outputs['location'][0]
forecast_version_basic_folder_location=df_inputs_outputs['location'][4]
client_data_folder_location=df_inputs_outputs['location'][1]
forecast_version=df_inputs_outputs['location'][2]
v_date=df_inputs_outputs['location'][3]
index_file_name='index_format_for_creating_forecast_jtmt_input_{}_{}'.format(forecast_version,v_date)

### פונקציות גלובליות


In [6]:
from functions import up_load_gdb, make_point, find_files_with_pattern, up_load_shp, up_load_df, split_index_by_taz, delete_and_add_by_TAZ

### העלת אזורי תנועה לחישוב


In [7]:
folder_path=r'{}\For_approval\Reference_tabels\shp'.format(client_data_folder_location)

gpd_name='tochnit_check.gdb'

forecast = up_load_gdb(r'{}\{}'.format(folder_path,gpd_name),'TAZ_211028_V3_Published_with_client_changes')

### הוספת מאפיינים גיאוגרפים לאזורי תנועה


In [8]:
forecast_point = make_point(forecast)

    # Load data layers
DISTRICT = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_DISTRICT')
urban = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_urban')
SCHOOLDISTRICT = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_SCHOOLDISTRICT')
PUMA = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_PUMA')
jerusalem_city = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_In_jerusal')
subdistrict_il = up_load_gdb(
        r'{}\background_files\subdistrict2008.gdb'.format(software_data_folder_location), 'subdistrict2008_ITM')
muni_JTMT = up_load_gdb(
        r'{}\background_files\MUNI_border.gdb'.format(software_data_folder_location), 'muni_under_JTMT_ITM')
    
# מחפש קבצים של jeru_metro_jtmt_border
jeru_metro_jtmt_border_files = find_files_with_pattern(r'{}\background_files'.format(software_data_folder_location), 'jeru_metro_jtmt_border')
filepath = ''

# לוקח את הקובץ של השכבה
for string in jeru_metro_jtmt_border_files:
    if string.endswith('.shp'):
        filepath = string

# מעלה את השכבה
jeru_metro_jtmt_border = up_load_shp(filepath)

# Geographical join between traffic zones and data layers
forecast_point_DISTRICT = forecast_point.sjoin(
        DISTRICT)[['Taz_num', 'puma2040_csv_DISTRICT']]
forecast_point_urban = forecast_point.sjoin(
        urban)[['Taz_num', 'BaseProjections2040_csv_urban']]
forecast_point_SCHOOLDISTRICT = forecast_point.sjoin(
        SCHOOLDISTRICT)[['Taz_num', 'puma2040_csv_SCHOOLDISTRICT']]
forecast_point_PUMA = forecast_point.sjoin(
        PUMA)[['Taz_num', 'puma2040_csv_PUMA']]
forecast_point_jerusalem_city = forecast_point.sjoin(
        jerusalem_city)[['Taz_num', 'jerusalem_city']]
forecast_point_subdistrict_il = forecast_point.sjoin(
        subdistrict_il[['geometry', 'ENG_NAME_nafa']])[['Taz_num', 'ENG_NAME_nafa']]
forecast_point_muni_JTMT = forecast_point.query('main_sector!="Palestinian"').sjoin(
        muni_JTMT[['Muni_Heb', 'Sug_Muni', 'CR_PNIM', 'geometry']])[['Taz_num', 'Muni_Heb', 'Sug_Muni', 'CR_PNIM']]
forecast_point_jeru_metro_jtmt_border = forecast_point.sjoin(
        jeru_metro_jtmt_border)[['Taz_num', 'jeru_metro']]

# Merge tables into one table
forecast = (forecast
                .merge(forecast_point_subdistrict_il, on='Taz_num', how='left')
                .merge(forecast_point_muni_JTMT, on='Taz_num', how='left')
                .merge(forecast_point_jeru_metro_jtmt_border, on='Taz_num', how='left')
                .merge(forecast_point_DISTRICT, on='Taz_num', how='left')
                .merge(forecast_point_urban, on='Taz_num', how='left')
                .merge(forecast_point_SCHOOLDISTRICT, on='Taz_num', how='left')
                .merge(forecast_point_PUMA, on='Taz_num', how='left')
                .merge(forecast_point_jerusalem_city, on='Taz_num', how='left'))

# Rename columns# Rename columns with flipped names
forecast.rename(columns={'ENG_NAME_nafa': 'zonetype'}, inplace=True)
forecast.rename(columns={'puma2040_csv_DISTRICT': 'DISTRICT'}, inplace=True)
forecast.rename(columns={'BaseProjections2040_csv_urban': 'urban'}, inplace=True)
forecast.rename(columns={'puma2040_csv_PUMA': 'PUMA'}, inplace=True)
forecast.rename(columns={'puma2040_csv_SCHOOLDISTRICT': 'SCHOOLDISTRICT'}, inplace=True)


# Data processing for feature columns
forecast.loc[forecast['main_sector'] == 'Palestinian', 'zonetype'] = 'Palestinian'
forecast['in_jerusalem_metropolin'] = 1
forecast.loc[forecast['jeru_metro'] == 0, 'in_jerusalem_metropolin'] = 0
forecast['yosh'] = 0
forecast.loc[forecast['zonetype'] == 'Judea and Samaria', 'yosh'] = 1

# Add constant columns
forecast['REGION'] = 1
forecast['slope'] = 0

forecast = forecast.set_index('Taz_num')
forecast['Taz_num'] = forecast.index

### שינוי עמודה Muni_Heb


In [9]:
forecast=change_Muni_Heb_to_Muni_Eng(software_data_folder_location, forecast)

### ייצוא שכבת אזורי תנועה לבקרת לקוח


In [10]:
save_shp_path=r'{}\For_approval\{}_taz_for_approval.shp'.format(client_data_folder_location,file_date)
    
col=['Taz_num','Name_hebre','Muni_Eng', 'main_sector', 'classification_name','Student_by_Classrooms','geometry']

forecast[col].to_file(save_shp_path,index=False,encoding='UTF-8')

C:\Users\gidon\AppData\Local\Temp\ipykernel_15712\3790716734.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  forecast[col].to_file(save_shp_path,index=False,encoding='UTF-8')


### מצב קיים לבקרה


In [11]:
sys.path.insert(0, r'{}'.format(create_forecast_basic_location))
from run_basic import run_notebook

folder_path=r'{}\For_approval\Reference_tabels'.format(client_data_folder_location)
pattern='TAZ_V'
matching_files=find_files_with_pattern(r'{}\shp'.format(folder_path), pattern)

forecast_2020=None

# אם יש שכבות חדשות
if len(matching_files) > 0:
        suffix = '.shp'
        filtered_files_strings = []
        for string in matching_files:
            if string.endswith(suffix):
                filtered_files_strings.append(string)

        filepath=filtered_files_strings[0]

        #load excel file
        workbook = load_workbook(filename=r"{}\current\inputs_outputs.xlsx".format(create_forecast_basic_location))

        #open workbook
        sheet = workbook.active

        #modify the desired cell
        sheet["B4"] = folder_path
        sheet["B5"] = True
        sheet["B6"] = filepath

        #save the file
        workbook.save(filename=r"{}\current\inputs_outputs.xlsx".format(create_forecast_basic_location))

        # # להריץ את קוד בסיס עם השכבות
        execution_result = run_notebook(r'{}\run_basic.ipynb'.format(create_forecast_basic_location))
        print("ad-hoc-Notebook execution result:", execution_result)

        if execution_result == True:
            df_inputs_outputs = pd.read_excel(r"{}\current\inputs_outputs.xlsx".format(create_forecast_basic_location))
            forecast_2020=up_load_df(r'{}'.format(df_inputs_outputs['location'][2]), r'2020_jtmt_forcast_full_{}_with_taz_changes'.format(file_date))
            
            col=[]

            forecast_2020['student_toddlers']=0
            forecast_2020['student_gov']=forecast_2020['student']
            forecast_2020['cbs_muni_student_left_by_pre_of_demand_left']=forecast_2020['student']
            forecast_2020['uni_students']=forecast_2020['univ']
            forecast_2020['emp_from_uni_student']=forecast_2020['emp_uni']
            forecast_2020['student_yeshiva']=forecast_2020['student_yeshiva_and_kollim']

            col_20=['Taz_num','Taz_name',
                'main_secto',
                'aprt_20', 'pop_without_dorms_yeshiva',
                'student_toddlers',
                'student_gov',
                'cbs_muni_student_left_by_pre_of_demand_left',
                'uni_students', 'student_dorms',
                'emp_from_uni_student',
                'student_yeshiva',
                'emp_okev',
                'emp_not_okev','student']
                 
            forecast_2020=pd.merge(forecast[col].reset_index(),forecast_2020[col_20],how='left',on='Taz_num').fillna(0)

            save_excel_path=r'{}\For_approval\{}_forecast_2020_For_approval.xlsx'.format(client_data_folder_location,file_date)

            forecast_2020[col_20].to_excel(save_excel_path,index=False)
# אם אין שכבות חדשות
else:
    forecast_2020=up_load_df(r'{}\background_files'.format(software_data_folder_location),'2020_jtmt_forcast_full_230720')

    col=[]

    # forecast_2020['student_toddlers']=0
    # forecast_2020['student_gov']=forecast_2020['student']
    # forecast_2020['cbs_muni_student_left_by_pre_of_demand_left']=forecast_2020['student']
    # forecast_2020['uni_students']=forecast_2020['univ']
    # forecast_2020['emp_from_uni_student']=forecast_2020['emp_uni']
    # forecast_2020['student_yeshiva']=forecast_2020['student_yeshiva_and_kollim']

    col_20=['Taz_num','Taz_name',
        'main_secto',
        'aprt_20', 'pop_without_dorms_yeshiva',
        'student_toddlers',
        'student_gov',
        'cbs_muni_student_left_by_pre_of_demand_left',
        'uni_students', 'student_dorms',
        'emp_from_uni_student',
        'student_yeshiva',
        'emp_okev',
        'emp_not_okev','student']

    forecast_2020=pd.merge(forecast[col].reset_index(),forecast_2020[col_20],how='left',on='Taz_num').fillna(0)

    save_excel_path=r'{}\For_approval\{}_forecast_2020_For_approval.xlsx'.format(client_data_folder_location,file_date)

    forecast_2020[col_20].to_excel(save_excel_path,index=False)

ad-hoc-Notebook execution result: True


### העלאת מרכיבי טבלת אינדקס


#### צריך להחליט האם להשתמש בגבול אינדקס או בעיבוד שכולל יצירת פוטנציאל


borders_index=up_load_shp(r'{}\For_approval\Reference_tabels\shp\gvul_index_with_ponten.shp'.format(client_data_folder_location))
path_to_upload=r'{}\For_approval\Reference_tabels'.format(client_data_folder_location)


In [12]:
borders_index=up_load_shp(r'{}\For_approval\Reference_tabels\shp\gvul_index.shp'.format(client_data_folder_location))
path_to_upload=r'{}\For_approval\Reference_tabels'.format(client_data_folder_location)

In [13]:
index=up_load_df(path_to_upload,index_file_name)
index=pd.merge(borders_index,index,on='id',how='right')

c:\Users\gidon\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### חלוקה לאזורי תנועה של התכניות


In [15]:
col=['add_uni_dorms',
    'add_old_age_home',
    'add_aprt',
    'Commerce_m2',
    'Business_m2',
    'Tourism_m2',
    'Public_m2',
    'Industry_m2',
    'emp_Public',
    'emp_Education',
    'emp_Commerce',
    'emp_Business',
    'emp_Industry',
    'emp_Tourism',
    'Classrooms',
    'add_uni_students']

index=split_index_by_taz(index,forecast,0.05,col)

### שכבת אינדקס


In [17]:
index=index.fillna(0)

promoteres_df = pd.read_excel(r'{}\promoteres.xlsx'.format(software_data_folder_location))

#מקדים לייצרת תעסוקה עוקב משקי בית
Industry_precent_per_hh=promoteres_df['value'][0]
Commerce_precent_per_hh=promoteres_df['value'][1]
Business_precent_per_hh=promoteres_df['value'][2]
Public_precent_per_hh=promoteres_df['value'][3]
Agriculture_precent_per_hh=promoteres_df['value'][4]
precent_emp_per_hh=promoteres_df['value'][5]

#מקדימים לייצרת מקומות עבודה מ"ר לפי ייעוד קרקע
m2_Industry_to_emp=promoteres_df['value'][6]
m2_Commerce_Hotel_to_emp=promoteres_df['value'][7]
m2_Business_to_emp=promoteres_df['value'][8]
m2_Public_to_emp=promoteres_df['value'][9]
m2_Agriculture_to_emp=promoteres_df['value'][10]
m2_Education_to_emp=promoteres_df['value'][11]
m2_Commerce_to_emp=m2_Commerce_Hotel_to_emp
m2_Tourism_to_emp=promoteres_df['value'][13]

#מילוי
old_age_home_fill=promoteres_df['value'][14]
uni_student_dorm_fill=promoteres_df['value'][15]

#מקדימי תעסוקה בעקבות חינוך
emp_education_per_student=promoteres_df['value'][16]
emp_Education_per_uni_student=promoteres_df['value'][17]
emp_Education_per_Yeshiva_student=promoteres_df['value'][18]


convert_dict={
'add_old_age_home': float,
'add_aprt': float,
'Commerce_m2': float,
'Business_m2': float,
'Tourism_m2': float,
'Public_m2': float,
'Industry_m2': float,
'emp_Public': float,
'emp_Education': float,
'emp_Commerce': float,
'emp_Business': float,
'emp_Industry': float,
'emp_Tourism': float,
'Classrooms': float,
'F2025': float,
'F2030': float,
'F2035': float,
'F2040': float,
'F2045': float,
'F2050': float,
'F2050_plus': float,
'Risk_factor': float,
'emp_fill_factor': float}

index = index.astype(convert_dict)

col_to_sum=['F2025',
'F2030',
'F2035',
'F2040']

index['precent_till_2040']=index[col_to_sum].sum(axis=1)

index['add_aprt_nominally']=index['add_aprt']

index['add_aprt']=index['add_aprt']*index['precent_till_2040']*index['Risk_factor']

index['Classrooms_nominally']=index['Classrooms']

index['Classrooms']=index['Classrooms']*index['precent_till_2040']*index['Risk_factor']

index['add_old_age_home_nominally']=index['add_old_age_home']

index['add_old_age_home']=index['add_old_age_home']*index['precent_till_2040']*index['Risk_factor']

index['add_uni_students_nominally']=index['add_uni_students']

index['add_uni_students']=index['add_uni_students']*index['precent_till_2040']*index['Risk_factor']

index['add_uni_dorms_nominally']=index['add_uni_dorms']

index['add_uni_dorms']=index['add_uni_dorms']*index['precent_till_2040']*index['Risk_factor']

list_category=['Commerce','Business','Industry','Tourism','Public']   #'Agriculture','Education','Public'
for c in list_category:
    index['{}_m2_nominally'.format(c)]=index['{}_m2'.format(c)]
    index['{}_m2'.format(c)]=index['{}_m2'.format(c)]*index['Risk_factor']*index['emp_fill_factor']*index['precent_till_2040']
    index['emp_{}_nominally'.format(c)]=index['emp_{}'.format(c)]
    index['emp_{}'.format(c)]=index['emp_{}'.format(c)]*index['precent_till_2040']*index['Risk_factor']*index['emp_fill_factor']
    index['add_emp_{}'.format(c)]=index['emp_{}'.format(c)]+index['{}_m2'.format(c)]/locals()['m2_{}_to_emp'.format(c)]

    


### ייצוא שכבת אינדקס לבקרת לקוח


In [19]:
index=index.drop(['geometry'], axis=1)


TypeError: unsupported operand type(s) for +: 'Timestamp' and 'Timestamp'

In [24]:
col=['id',
'add_aprt',
'add_aprt_nominally',
'add_old_age_home',
'add_old_age_home_nominally',
'add_uni_dorms',
'add_uni_dorms_nominally',
'add_uni_students',
'add_uni_students_nominally',
'Classrooms','Classrooms_nominally',
'Commerce_m2',
'Commerce_m2_nominally',
'add_emp_Commerce',
'Tourism_m2',
'Tourism_m2_nominally',
'add_emp_Tourism',
'Business_m2',
'Business_m2_nominally',
'add_emp_Business',
'Public_m2',
'Public_m2_nominally',
'add_emp_Public',
'Industry_m2',
'Industry_m2_nominally',
'add_emp_Industry']

save_excel_path=r'{}\For_approval\Reference_tabels\{}_index_2040_{}_For_approval.xlsx'.format(client_data_folder_location,file_date,forecast_version)
index.reset_index()[col].pivot_table(index='id',aggfunc=sum).to_excel(save_excel_path,index=True)

### חישוב תחזית


### הוספת תוספת בעקבות האינדקס


כנראה צריך למחוק את זה


#מקדים לייצרת תעסוקה עוקב משקי בית
Industry_precent_per_hh=0
Commerce_precent_per_hh=0.55
Business_precent_per_hh=0.2
Public_precent_per_hh=0.25
Agriculture_precent_per_hh=0

precent_emp_per_hh=0.0

#מקדימים לייצרת מקומות עבודה מ"ר לפי ייעוד קרקע
m2_Industry_to_emp=200
m2_Commerce_Hotel_to_emp=30
m2_Business_to_emp=30
m2_Public_to_emp=60
m2_Agriculture_to_emp=0
m2_Education_to_emp=0
m2_Commerce_to_emp=m2_Commerce_Hotel_to_emp
m2_Tourism_to_emp=100

#מילוי

old_age_home_fill=1.5
uni_student_dorm_fill=3

#מקדימי תעסוקה בעקבות חינוך

emp_education_per_student=3.75
emp_Education_per_uni_student=0.1
emp_Education_per_Yeshiva_student=0.10


In [25]:
col=['Taz_num','id',
    'add_aprt',
    'add_aprt_nominally',
    'add_old_age_home',
    'add_old_age_home_nominally',
    'add_uni_dorms',
    'add_uni_dorms_nominally',
    'add_uni_students',
    'add_uni_students_nominally',
    'Classrooms','Classrooms_nominally',
    'Commerce_m2',
    'Commerce_m2_nominally',
    'add_emp_Commerce',
    'Tourism_m2',
    'Tourism_m2_nominally',
    'add_emp_Tourism',
    'Business_m2',
    'Business_m2_nominally',
    'add_emp_Business',
    'Public_m2',
    'Public_m2_nominally',
    'add_emp_Public',
    'Industry_m2',
    'Industry_m2_nominally',
    'add_emp_Industry']
index=index[col].pivot_table(index='Taz_num', aggfunc='sum').fillna(0)

In [26]:

col=[ 'add_aprt','add_uni_dorms', 'add_emp_Business',
'add_emp_Commerce',
    'add_emp_Industry',
    'add_emp_Public',
    'add_emp_Tourism','add_uni_students','add_old_age_home','Classrooms']

forecast=forecast.merge(index[col],left_index=True,right_index=True,how='left')

forecast=forecast.fillna(0)

col=['aprt_20','student','uni_students','student_dorms','student_yeshiva','emp_not_okev']

forecast_2020=forecast_2020.set_index('Taz_num')

forecast=forecast.merge(forecast_2020[col],left_index=True,right_index=True,how='left')

forecast=forecast.rename(columns={'student':'student_20','uni_students':'uni_students_20','student_dorms':'student_dorms_20','student_yeshiva':'student_yeshiva_and_kollim_20','emp_not_okev':'emp_not_okev_20'})

age_des_types=up_load_df(r'{}\background_files'.format(software_data_folder_location),'age_des_types')

forecast=forecast.merge(age_des_types,on='classification_name',how='left').fillna(0)

#### יח"ד של השכונה ויצירת אנשים לפי קטלוג
forecast['aprt']=forecast['aprt_20']+forecast['add_aprt']

forecast['pop_without_dorms_yeshiva']=forecast['aprt']*forecast['hh_size']

#### תלמידים בעקבות האוכלוסיה
forecast['student_demand_pre']=forecast['pop_0']/5*2+forecast['pop_5']+forecast['pop_10']+forecast['pop_15']/5*3+forecast['pop_0']/5*3*0.5

forecast['student_demand']=forecast['student_demand_pre']*forecast['pop_without_dorms_yeshiva']


In [27]:


forecast['student_to_fill_demand']=forecast['student_demand']-forecast['student_20']

forecast.loc[forecast['student_to_fill_demand']<0,'student_to_fill_demand']=0


In [28]:

forecast['student']=forecast['student_to_fill_demand']+forecast['student_20']

forecast.loc[forecast['Student_by_Classrooms']==1,'student']=forecast['Classrooms']*30

forecast.loc[forecast['Student_by_Classrooms']==1,'student']=forecast['Classrooms']*30

#### תעסוקה בעקבות תלמידים
forecast['emp_from_student']=forecast['student']/emp_education_per_student

#### סטודנטים
forecast['student_dorms']=forecast['add_uni_dorms']*uni_student_dorm_fill+forecast['student_dorms_20']

#### מספר הסטודנטים יהיה בהתאם לגודל של הקיים
forecast['uni_students']=forecast['uni_students_20']+forecast['add_uni_students']

#### תעסוקה בעקבות סטודנטים
forecast['emp_from_uni_student']=forecast['uni_students']*emp_Education_per_uni_student

#### תלמידי ישיבה ותעסוקה מישיבה
forecast['student_yeshiva_and_kollim']=forecast['student_yeshiva_and_kollim_20']*1.15 #גידול מינורי

forecast['emp_from_Yeshiva_student']=forecast['student_yeshiva_and_kollim']*emp_Education_per_Yeshiva_student

forecast['emp_Education']=forecast['emp_from_student']+forecast['emp_from_Yeshiva_student']+forecast['emp_from_uni_student']

#### תעסוקה לא עוקב
#### מקומות עבודה תעשייה

forecast['Indus']=forecast['add_emp_Industry']+forecast['emp_not_okev_20']*0.7 #חלוקת מצב הקיים הערכה 

#### מקומות עבודה מסחר ומלונאות
forecast['Com_hotel']=forecast['add_emp_Commerce']+forecast['add_emp_Tourism']+forecast['emp_not_okev_20']*0.2 #חלוקת מצב הקיים הערכה 

#### מקומות עבודה משרדים
forecast['Business']=forecast['add_emp_Business']+forecast['emp_not_okev_20']*0.1 #חלוקת מצב הקיים הערכה 

forecast['agri']=0

forecast['Public']=0

#### מקומות עבודה עוקב משקי בית 
forecast['emp_okev']=forecast['aprt']*precent_emp_per_hh

#### מקומות עבודה עוקב אוכלוסייה
list_category=[	'Com_hotel','Business','Indus','Public'] 
list_category_index=['Commerce','Business','Industry','Public'] 

for c,i in zip(list_category, list_category_index):
    forecast['{}'.format(c)]= forecast['{}'.format(c)].fillna(0)+forecast['emp_okev']*locals()['{}_precent_per_hh'.format(i)]

#### סך מקומות עבודה
col_to_sum_emp=['Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education','agri']

forecast['total_emp']=forecast[col_to_sum_emp].sum(axis=1)

#### המרת התפלגות גילים מאחוזים למספרים מוחלטים
col=['pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up']

forecast[col]=forecast[col].multiply(forecast['pop_without_dorms_yeshiva'], axis="index")

#### הוספת סטודנטים למשקי הבית, אוכלוסיה ותפלגות גילים
forecast['pop']=forecast['pop_without_dorms_yeshiva']+forecast['student_dorms']

forecast['pop_20']=forecast['pop_20']+forecast['student_dorms']*0.6

forecast['pop_25']=forecast['pop_25']+forecast['student_dorms']*0.4

forecast['hh']=forecast['aprt']+forecast['student_dorms']/uni_student_dorm_fill

#### הוספת דיור מוגן למשקי הבית, אוכלוסיה ותפלגות גילים
forecast['pop']=forecast['pop']+forecast['add_old_age_home']*old_age_home_fill

forecast['pop_75up']=forecast['pop_75up']+forecast['add_old_age_home']*old_age_home_fill

forecast['hh']=forecast['hh']+forecast['add_old_age_home']

#### יצירת עמודת יוצאים לעבודה מתוך האוכלוסייה שגרה
pre_woman=0.5

pre_man=1-pre_woman

work_age=[ 'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60']

under_work_age=[ 'pop_15', 'pop_20']

over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']

forecast['work_age']=forecast[work_age].sum(axis=1)

forecast['under_work_age']=forecast[under_work_age].sum(axis=1)

forecast['over_work_age']=forecast[over_work_age].sum(axis=1)

sector='U_Orthodox'

forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['work_age']*pre_woman*0.75+forecast['work_age']*pre_man*0.44
forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['pop_emp']+forecast['under_work_age']*pre_woman*0.07+forecast['under_work_age']*pre_man*0.09
forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['pop_emp']+forecast['over_work_age']*pre_woman*0.05+forecast['over_work_age']*pre_man*0.09

sector='Jewish'

forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['work_age']*pre_woman*0.84+forecast['work_age']*pre_man*0.87
forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['pop_emp']+forecast['under_work_age']*pre_woman*0.20+forecast['under_work_age']*pre_man*0.15
forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['pop_emp']+forecast['over_work_age']*pre_woman*0.05+forecast['over_work_age']*pre_man*0.09

sector='Arab'

forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['work_age']*pre_woman*0.23+forecast['work_age']*pre_man*0.78
forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['pop_emp']+forecast['under_work_age']*pre_woman*0.2+forecast['under_work_age']*pre_man*0.15
forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['pop_emp']+forecast['over_work_age']*pre_woman*0.05+forecast['over_work_age']*pre_man*0.09

sector='arabs_behined_seperation_wall'

forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['work_age']*pre_woman*0.23+forecast['work_age']*pre_man*0.78
forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['pop_emp']+forecast['under_work_age']*pre_woman*0.2+forecast['under_work_age']*pre_man*0.15
forecast.loc[forecast['main_sector']==sector,'pop_emp']=forecast['pop_emp']+forecast['over_work_age']*pre_woman*0.05+forecast['over_work_age']*pre_man*0.09

## חישוב אבטלה
forecast['pop_emp_employed']=0

forecast.loc[forecast['main_sector']=="U_Orthodox",'pop_emp_employed']=forecast['pop_emp']*0.95

forecast.loc[forecast['main_sector']=="Jewish",'pop_emp_employed']=forecast['pop_emp']*0.96

arab_sector=['arabs_behined_seperation_wall','Arab']

forecast.loc[forecast['main_sector'].isin(arab_sector),'pop_emp_employed']=forecast['pop_emp']*0.98

#### הוספת תלמידי ישיבה  למשקי הבית, אוכלוסיה ותפלגות גילים
forecast['hh']=forecast['hh']+forecast['student_yeshiva_and_kollim']

forecast['pop']=forecast['pop']+forecast['student_yeshiva_and_kollim']

forecast['pop_15']=forecast['pop_15']+forecast['student_yeshiva_and_kollim']*0.7

forecast['pop_20']=forecast['pop_20']+forecast['student_yeshiva_and_kollim']*0.3

### ייצוא תוצאות


In [29]:
col = [
        'Taz_num', 
        'Name_hebre',
        'Muni_Eng', 
        'main_sector', 
        'classification_name', 
        'aprt_20', 
        'add_aprt', 
        'aprt', 
        'hh_size', 
        'pop_without_dorms_yeshiva', 
        'pop_emp_employed', 
        'student_20', 
        'student', 
        'uni_students_20', 
        'add_uni_students', 
        'uni_students', 
        'student_dorms_20', 
        'add_uni_dorms', 
        'student_dorms', 
        'student_yeshiva_and_kollim', 
        'add_old_age_home', 
        'emp_from_student', 
        'emp_from_uni_student', 
        'emp_from_Yeshiva_student', 
        'emp_Education', 
        'emp_okev', 
        'add_emp_Business', 
        'add_emp_Commerce', 
        'add_emp_Industry', 
        'add_emp_Public', 
        'add_emp_Tourism', 
        'total_emp'
    ]

save_excel_path = r'{}\For_approval\{}_forecast_2040_{}_for_approval.xlsx'.format(client_data_folder_location, file_date, forecast_version)

forecast[col].to_excel(save_excel_path, index=False)

BaseProjections2040 = pd.read_csv(r'{}\background_files\BaseProjections2040.csv'.format(software_data_folder_location))
puma2040 = pd.read_csv(r'{}\background_files\puma2040.csv'.format(software_data_folder_location))


In [30]:

forecast.loc[forecast['Taz_num'] < 7001, 'AGG_TAZ'] = forecast['Taz_num'] // 100
forecast.loc[forecast['Taz_num'] >= 7001, 'AGG_TAZ'] = forecast['Taz_num'] // 10

forecast.rename(columns={'Taz_num': 'TAZ'}, inplace=True)

save_excel_path = r'{}\{}_puma2040_{}.csv'.format(client_data_folder_location, file_date,forecast_version)

delete_and_add_by_TAZ(forecast, puma2040).to_csv(save_excel_path, index=False)

forecast_col =[
         'TAZ', 
         'Muni_Eng',
         'yosh', 
         'in_jerusalem_metropolin', 
         'jerusalem_city', 
         'main_sector', 
         'hh', 
         'pop', 
         'pop_0', 
         'pop_5', 
         'pop_10',
         'pop_15', 
         'pop_20', 
         'pop_25', 
         'pop_30', 
         'pop_35', 
         'pop_40', 
         'pop_45', 
         'pop_50', 
         'pop_55', 
         'pop_60', 
         'pop_65', 
         'pop_70',
         'pop_75up', 
         'total_emp', 
         'Indus', 
         'Com_hotel', 
         'Business', 
         'Public', 
         'emp_Education', 
         'agri', 
         'student', 
         'uni_students',
         'student_yeshiva_and_kollim', 
         'pop_emp_employed', 
         'slope', 
         'urban'
      ] 

format_needed_col = [
      'TAZ', 
      'Muni_Eng',
      'yosh', 
      'in_jerusalem_metropolin', 
      'jerusalem_city', 
      'sector', 
      'hh_total', 
      'pop', 
      'age0_4', 
      'age5_9', 
      'age10_14', 
      'age15_19', 
      'age20_24', 
      'age25_29', 
      'age30_34', 
      'age35_39', 
      'age40_44', 
      'age45_49', 
      'age50_54', 
      'age55_59', 
      'age60_64', 
      'age65_69', 
      'age70_74', 
      'age75up', 
      'emp_tot', 
      'indus', 
      'com_hotel', 
      'business', 
      'public', 
      'education', 
      'agri', 
      'student', 
      'univ', 
      'UO_Hi_Ed', 
      'pop_emp_employed', 
      'slope', 
      'urban'
      ]


forecast_for_model = forecast[forecast_col]
forecast_for_model.columns = format_needed_col

save_excel_path = r'{}\{}_BaseProjections2040_{}.csv'.format(client_data_folder_location, file_date, forecast_version)
delete_and_add_by_TAZ(forecast_for_model, BaseProjections2040).to_csv(save_excel_path, index=False)